In [ ]:
import skimage as sk
from skimage.io import imread, imshow, imsave
import numpy as np
import os
from glob import glob
import napari
from cellpose import models, io, utils
import pandas as pd
import pyclesperanto as cle
from imaris_ims_file_reader.ims import ims
from tqdm import tqdm

In [ ]:
ims_files = sorted(glob('E:/ParadaKusz_Lab/OIC-153_Fluorescence-quantification-in-HUVEC-cells/H2DCFDA_05292025/**/*.ims',recursive=True))
imgs = [ims(file,ResolutionLevelLock=0,write=True) for file in ims_files]


In [ ]:
all_img = [imgs[i][0,0,:,:,:] for i in range(len(imgs))]

In [ ]:
import random
nums = range(len(all_img))
i = random.randint(min(nums),max(nums))
print(i)

In [ ]:
test_image = all_img[i]
test_image.shape

In [ ]:
viewer = napari.view_image(test_image)

In [ ]:
from cellpose import denoise

In [ ]:
dn = denoise.DenoiseModel(model_type="denoise_cyto3", gpu=True)
imgs_dn = dn.eval(all_img, channels=None, diameter=50.)
imgs_dn = [np.squeeze(img) for img in imgs_dn]

In [ ]:
imgs_dn.shape

In [ ]:
def normalize_images(input_image,tophat_radius):
    input_gpu = cle.push(input_image)
    #normalizing the image stack
    equalized_intensities_stack = cle.create_like(input_gpu)
    a_slice = cle.create([input_gpu.shape[1], input_gpu.shape[2]])
    num_slices = input_gpu.shape[0]
    mean_intensity_stack = cle.mean_of_all_pixels(input_gpu)
    corrected_slice = None
    for z in range(0, num_slices):
        # get a single slice out of the stack
        cle.copy_slice(input_gpu, a_slice, z)
        # measure its intensity
        mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
        # correct the intensity
        correction_factor = mean_intensity_slice/mean_intensity_stack
        corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
        # copy slice back in a stack
        cle.copy_slice(corrected_slice, equalized_intensities_stack, z)
    #background subtraction (increase the signal to noise ratio for improved segmentation results)
    background_subtracted_top_hat = cle.top_hat_sphere(equalized_intensities_stack,radius_x=tophat_radius,radius_y=tophat_radius,radius_z=tophat_radius)
    #pull data off gpu
    input_pull = cle.pull(input_gpu)
    background_subtracted_top_hat_pull = cle.pull(background_subtracted_top_hat)
    equalized_intensities_stack_pull = cle.pull(equalized_intensities_stack)
    return background_subtracted_top_hat_pull

In [ ]:
cle.select_device("NVIDIA")

In [ ]:
io.logger_setup()
model = models.CellposeModel(gpu=True,model_type="cyto3") # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _ = model.eval(imgs_dn, channels=channels, flow_threshold=0.4,cellprob_threshold=0.0, diameter=60)


In [ ]:
imgs_stack = np.stack(all_img)
dn_imgs_stack = np.stack(imgs_dn)
masks_stack = np.stack(masks)

In [ ]:
viewer = napari.view_image(imgs_stack,name='dn')
viewer.add_image(dn_imgs_stack,name='denoise')
viewer.add_labels(masks_stack,name='masks')

In [ ]:
corrected = normalize_images(imgs_stack,tophat_radius=10)

In [ ]:
io.logger_setup()
model = models.CellposeModel(gpu=True,model_type="cyto3") # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _ = model.eval(corrected, channels=channels, flow_threshold=0.4,cellprob_threshold=0.0, diameter=60)


In [ ]:
masks_stack = np.stack(masks)

In [ ]:
viewer = napari.view_image(corrected)
viewer.add_labels(masks, name='masks')

In [ ]:
props = sk.measure.regionprops_table(masks,properties=('label','area'))

In [ ]:
props